<a href="https://colab.research.google.com/github/jmcgonegal/colab/blob/master/Imagenet_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Convolution2D, MaxPooling2D, Activation, BatchNormalization, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping

# path to the model weights files.
# dimensions of our images.
img_width, img_height = 384, 384

train_data_dir = 'train2' # directory with images and 
validation_data_dir = 'validation'
epochs = 2000
batch_size = 3 # how many images to train in the batch (limited by size of memory)
steps_per_epoch=200 # training steps
validation_steps=75 # validation steps
class_mode='categorical'

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    #rotation_range=5,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    rescale=1. / 255,
    #shear_range=0.1,
    #zoom_range=0.2,
    #horizontal_flip=True, 
    fill_mode='nearest',
    validation_split=0.2
)

# test_datagen = ImageDataGenerator(rescale=1. / 255)

# images we will use for training
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode='rgb',
    class_mode=class_mode,
    subset='training',
    shuffle=True)

# images we will use for validation
validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode='rgb',
    subset='validation',
    class_mode=class_mode)

# classes found for training
class_dictionary = train_generator.class_indices
num_classes = len(class_dictionary)

# our image shape width,height,rgb
input_shape = (img_width, img_height, 3)

# build the VGG16/VGG19 network
vgg_model = applications.VGG19(weights='imagenet', include_top=False, input_shape=input_shape)

# dont adjust weights of these layers
for i in range(len(vgg_model.layers)):
    vgg_model.layers[i].trainable = False
    
# build our training model
top_model = Sequential()
top_model.add(vgg_model)
top_model.add(Flatten(input_shape=input_shape))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(num_classes, activation='sigmoid'))

top_model.summary()

#filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
filepath="weights.best.hdf5"

# only do this if file exists?
#top_model.load_weights(filepath)

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
top_model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(lr=1e-5), 
                  metrics=['accuracy'])

print('Model loaded')

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=batch_size, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='epoch')
early_stop = EarlyStopping(monitor='loss', patience=5, verbose=1)

callbacks_list = [checkpoint, tensorboard] # early_stop,

In [0]:
# fit the model
history = top_model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=callbacks_list,
    max_queue_size=50,
    workers=4,
    use_multiprocessing=False)